In [ ]:
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from skimage.transform import resize
import scipy.stats as stats
import matplotlib.image
from matplotlib import pyplot as plt
from matplotlib import colors
from PIL import Image
from scipy.stats import pearsonr
import h5py
import scipy.io as sio

folder_path = "/path/to/your/data/"
# The V4 map data can be downloaded from: https://zenodo.org/records/10972034

# V4 digital twin image preference map

In [ ]:
# Visualize the top-9 images of each grid in the V4 digital twin
features = np.load(folder_path + "V4DT/PRsp.npy")
features = np.transpose(features, (2, 1, 0)) # (128, 128, 50000)
features = np.swapaxes(features, 0, 1)
grid_num = int(features.shape[0])
roi = np.load(folder_path + "V4DT/ROI.npy").T # 3048 roi voxels
# define the size of a single image
img_size = 30
line_width = 5
top_img_num = 9
# create a blank map of black color (R=0, G=0, B=0)
map = np.zeros((grid_num * (img_size*3 + line_width) + line_width,
                grid_num * (img_size*3 + line_width) + line_width, 
                3))
grid_top_images = np.zeros((grid_num, grid_num, top_img_num)) # store the top 9 images of each grid
# fill the map with the images
for i in tqdm(range(grid_num), desc="map initialization...", disable=False):
    for j in range(grid_num):
            if roi[i, j] == 1:
                # 1-indexed image names
                image_label = np.arange(50000) + 1 # or "labels + 1" if features were selected from the top-3 responsive images
                # sort the mean responses (from small to large) and the image_label according to the order of mean responses
                _, image_label = zip(*sorted(zip(features[i, j, :],image_label)))
                # take the top nine images with largest mean response
                image_label = np.flip(image_label[-top_img_num:])
                # store the 0-indexed 9 imgs of each grid
                grid_top_images[i, j, :] = (image_label - 1).astype(int)
                # locate the top left corner of the current grid in the map
                x = i * (img_size*3 + line_width) + line_width
                y = j * (img_size*3 + line_width) + line_width
                # fill the map's current grid with the selected nine images
                for row in range(3):
                    for col in range(3):    
                        # load the image
                        path = folder_path + "50K_Imgset/" + str(int(image_label[row*3+col])) + ".bmp" # the image name is 1-indexed
                        img = np.array(Image.open(path))[20:80, 20:80, :] # obtain the non-blurred central part of the image
                        img = resize(img, (img_size, img_size, 3), anti_aliasing=True) # resize the image
                        img = np.fliplr(np.flipud(img)) # flip the image vertically and horizontally
                        # put the image onto the map
                        map[x + (2 - row) * img_size : x + ((2 - row) + 1) * img_size, 
                            y + (2 - col) * img_size : y + ((2 - col) + 1) * img_size, 
                            :] = img
            else:
                # fill the map's current grid with pure white color
                x = i * (img_size*3 + line_width)
                y = j * (img_size*3 + line_width)
                map[x : x + img_size*3 + line_width*2, y : y + img_size*3 + line_width*2, :] = 1.0
map = np.fliplr(np.flipud(map)) # top-bottom flip and then left-right flip
size = (img_size*3 + line_width)
map = map[38*size:114*size, 38*size:105*size, :] # only keep the roi part
map_save_path = folder_path + "Fig1/V4_DT_full.bmp"
np.save(folder_path + "V4DT/top9_0index.npy", grid_top_images)
matplotlib.image.imsave(map_save_path, map)
del features, map

In [ ]:
# reduce map size
image = Image.open(folder_path + "Fig1/V4_DT_full.bmp")
width, height = image.size
factor = 3
resized_image = image.resize((int(np.round(width/factor)), int(np.round(height/factor))), Image.Resampling.LANCZOS)
resized_image.save(folder_path + "Fig1/V4_DT.jpg")

In [ ]:
# visualization of RSOM training from V4 digital twin neuronal columns' tuning curves + estimated retinotopic positions
# visualize the V4 data ROI shape
roi = np.load(folder_path + "V4DT/ROI.npy").T # (128, 128) <class 'numpy.ndarray'>
roi = np.flip(roi)
roi = roi[37:115, 37:106]

fig, ax = plt.subplots(figsize=(6, 6))
cmap = colors.ListedColormap(['white', 'lightyellow'])  # 0 for white, 1 for lightyellow
ax.imshow(roi, cmap=cmap, interpolation='none')
roi_grid = np.ma.masked_where(roi == 0, roi) # Create a masked array to only apply grid where roi is 1
# Add grid lines only for the region of interest
for i in range(roi.shape[0]):
    for j in range(roi.shape[1]):
        if roi[i, j] == 1:  # Only show grid for ROI entries with value 1
            ax.plot([j-0.5, j+0.5], [i-0.5, i-0.5], color='black', linestyle='--', linewidth=0.5)  # Top line
            ax.plot([j-0.5, j+0.5], [i+0.5, i+0.5], color='black', linestyle='--', linewidth=0.5)  # Bottom line
            ax.plot([j-0.5, j-0.5], [i-0.5, i+0.5], color='black', linestyle='--', linewidth=0.5)  # Left line
            ax.plot([j+0.5, j+0.5], [i-0.5, i+0.5], color='black', linestyle='--', linewidth=0.5)  # Right line
# Add contour lines for the region of interest with a 3D effect (using multiple levels and shadowing)
contour = ax.contour(roi, levels=[0.5], colors='black', linewidths=2, alpha=0.9)
# Adding additional contour for 3D effect (shadow)
ax.contour(roi, levels=[0.5], colors='gray', linewidths=6, alpha=0.75, linestyles='solid')
# Remove axis labels and ticks
ax.set_xticks([])
ax.set_yticks([])
# Remove the axis box
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
# Show the plot
plt.tight_layout()
plt.savefig(folder_path + "Fig1/ROI.png", dpi=1000)
plt.close()

# Create a 60 by 60 grid to visualize the SOM map
rows, cols = 60, 60
# Create figure and axes
fig, ax = plt.subplots(figsize=(4, 4))  # Adjust figsize for higher resolution
# Plot the grid
for x in range(rows + 1):
    ax.plot([x, x], [0, cols], color='black', linewidth=0.5)
for y in range(cols + 1):
    ax.plot([0, rows], [y, y], color='black', linewidth=0.5)
# Set the aspect of the plot to be equal
ax.set_aspect('equal')
# Set limits to match grid size
ax.set_xlim(0, rows)
ax.set_ylim(0, cols)
# Turn off axes ticks
ax.set_xticks([])
ax.set_yticks([])
# Save as high resolution
plt.savefig(folder_path + "Fig1/grids.png", dpi=3000)  # Save the image in high resolution

# V4 digital twin shape-texture image preference

This subsection is contributed by Yingjue Bian

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    # just use CPU
import tensorflow as tf
from tensorflow.keras import applications, regularizers, optimizers, activations
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.layers import Input, Dense, Dropout 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Cropping2D,ReLU
from tensorflow.keras.layers import SeparableConv2D, Add, Reshape, Permute, LocallyConnected2D, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
# import tensorflow as tf
import os
import numpy as np
from PIL import Image
import scipy.io as sio

def load_and_resize_images(folder_path, target_size=(50, 50)):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            img_path = os.path.join(folder_path, filename)
            try:
                img = Image.open(img_path).resize(target_size) # grayscale images
                # print(img)
                # 转换为RGB格式，确保无论格式如何都有三通道
                img = img.convert("RGB")
                img_array = np.array(img)  # 转换为NumPy数组，形状为 (100, 100, 3)
                # print(img_array)
                # Create a new white canvas of (100, 100)
                padded_image = np.full((100, 100, 3), 255, dtype=np.uint8)
                # Calculate padding offsets
                x_offset = (100 - target_size[0]) // 2
                y_offset = (100 - target_size[1]) // 2
    # Place the resized image onto the white canvas
                padded_image[y_offset:y_offset + target_size[0], x_offset:x_offset + target_size[1]] = img_array
                images.append(padded_image)  # 将处理后的NumPy数组添加到列表中
            except IOError:
                print(f"无法读取图片文件: {filename}")
    # 将所有图片堆叠成四维数组
    return np.array(images)


# 使用方法
texture_path = r"TextureStimuli"
shape_path = r"ShapeStimuli"

textureimages = load_and_resize_images(texture_path, target_size=(50, 50))
shapeimages = load_and_resize_images(shape_path, target_size=(50, 50))
shapeimageset = load_and_resize_images(shape_path, target_size=(100, 100))
textureimageset = load_and_resize_images(texture_path, target_size=(100, 100))

shape_num = shapeimages.shape[0]
texture_num = textureimages.shape[0]

images = np.concatenate([textureimages, shapeimages], axis=0)
imageset = np.concatenate([textureimageset, shapeimageset], axis=0)

sio.savemat('imgset.mat', {'imgset': imageset.astype('float32')/255})

num = images.shape[0]

# print(num)
#%% model setting

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

image_input = Input(shape=(100,100,3))
x = Conv2D(64, (5, 5),strides=(1,1))(image_input)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = Dropout(0.1)(x)

x = Conv2D(100, (3, 3),strides=(1,1))(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = SeparableConv2D(100,(3,3),strides=(1,1))(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = Dropout(0.1)(x)

residual = Conv2D(200, (1, 1), strides=(1, 1), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(200,(3,3),strides=(1,1), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = SeparableConv2D(200,(3,3),strides=(1,1), padding='same')(x)
x = Add()([x, residual])
x = Activation(activations.sigmoid)(x)
x = Dropout(0.1)(x)

residual = Conv2D(400, (1, 1), strides=(1, 1), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(400,(3,3),strides=(1,1), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = SeparableConv2D(400,(3,3),strides=(1,1), padding='same')(x)
x = Add()([x, residual])
x = MaxPooling2D((2, 2))(x)
x = Activation(activations.sigmoid)(x)
x = Dropout(0.1)(x)

residual = x
x = SeparableConv2D(400,(3,3),strides=(1,1), padding='same')(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = SeparableConv2D(400,(3,3),strides=(1,1), padding='same')(x)
x = Add()([x, residual])
x = Activation(activations.sigmoid)(x)
x = Dropout(0.1)(x)
x = Cropping2D(cropping=((2, 2), (2, 2)))(x)

x = Reshape((49,20,20))(x)
x = Permute((2, 3, 1))(x)
x = Conv2D(32, (3, 3),strides=(1,1))(x)
x = BatchNormalization()(x)
x = Activation(activations.sigmoid)(x)
x = LocallyConnected2D(16, (3,3), implementation =1)(x)
x = Activation(activations.sigmoid)(x)
x = Conv2D(64, (3, 3),strides=(1,1), padding='same')(x)

x = Conv2DTranspose(1,(8,8),strides = (8,8))(x)
x = LocallyConnected2D(1, (1,1), implementation =3)(x)
x = Flatten()(x)

model = Model(inputs = image_input, outputs = x)

### The Conv2DTranspose is only used for reshaping the feature map
w1 = model.layers[-3].get_weights()
w2 = np.zeros((8, 8, 1, 64))
ni = 0
for i in range(8):
    for j in range(8):
        w2[i,j,0,ni] = 1
        ni = ni+1
w1[0] = w2
w1[1][0] = 0
model.layers[-3].set_weights(w1)
model.layers[-3].trainable = False

# weight = h5py.File()
# print(model.summary())
model.load_weights(folder_path + "V4DT/best_model.hdf5")

# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#%% inference
# import gc

# batch_size = 2
# num_batches = len(images) // batch_size
# PRsp_list = []
# for i in range(num_batches):
#     batch = images[i * batch_size : (i + 1) * batch_size]
#     batch_output = model.predict(batch)
#     PRsp_list.append(batch_output)
#     # tf.keras.backend.clear_session()  
#     del batch, batch_output  
#     gc.collect() 

# if len(images) % batch_size != 0:
#     batch = images[num_batches * batch_size:]
#     batch_output = model.predict(batch)
#     PRsp_list.append(batch_output)
#     del batch, batch_output 
#     gc.collect()

# PRsp = np.concatenate(PRsp_list, axis=0)
# # PRsp = np.reshape(PRsp,(num,128,128)) 
PRsp = model.predict(images,batch_size=2) # 需要分batch处理，否则预测错误

PRsp = np.reshape(PRsp,(num,128,128))
sio.savemat('PRsp.mat', {'PRsp':PRsp})

In [ ]:
# Visualize the top-9 shape and texture images of each grid in the V4 digital twin
STimg = sio.loadmat(folder_path + "V4DT/ST_imgset.mat")["imgset"] # (814, 100, 100, 3): 448 texture + 366 shape image stimuli
# (814, 128, 128): 128 by 128 V4 digital twin responses to 814 shape and texture image stimuli
features = sio.loadmat(folder_path + "V4DT/ST_PRsp.mat")["PRsp"]
grid_num = int(features.shape[1])
stmap = np.ones((grid_num, grid_num)) # 1 for shape preferring, 2 for texture preferring
assert grid_num == features.shape[2]
roi = np.load(folder_path + "V4DT/ROI.npy").T # 3048 roi voxels
# define the size of a single image
img_size = 30
line_width = 5
top_img_num = 9
# create a blank map of black color (R=0, G=0, B=0)
map = np.zeros((grid_num * (img_size*3 + line_width) + line_width,
                grid_num * (img_size*3 + line_width) + line_width, 
                3))
# fill the map with the images
for i in tqdm(range(grid_num), desc="map initialization...", disable=False):
    for j in range(grid_num):
            if roi[i, j] == 1:
                image_label = np.arange(814) # 0-indexed top preferred images
                # sort the mean responses (from small to large) and the image_label according to the order of mean responses
                _, image_label = zip(*sorted(zip(features[:, i, j], image_label)))
                # take the top nine images with largest mean response
                image_label = np.flip(image_label[-top_img_num:])
                # locate the top left corner of the current grid in the map
                x = i * (img_size*3 + line_width) + line_width
                y = j * (img_size*3 + line_width) + line_width
                # fill the map's current grid with the selected nine images
                texture_img_count = 0
                for row in range(3):
                    for col in range(3):    
                        # load the image
                        img = STimg[int(image_label[row*3+col])][20:80, 20:80, :] # obtain the non-blurred central part of the image
                        img = resize(img, (img_size, img_size, 3), anti_aliasing=True) # resize the image
                        img = np.fliplr(np.flipud(img)) # flip the image vertically and horizontally
                        # put the image onto the map
                        map[x + (2 - row) * img_size : x + ((2 - row) + 1) * img_size, 
                            y + (2 - col) * img_size : y + ((2 - col) + 1) * img_size, 
                            :] = img
                        if image_label[row*3+col] < 448: texture_img_count += 1 # texture image
                if texture_img_count > 4: stmap[i, j] = 2 # the current pixel is texture preferring
            else:
                stmap[i, j] = 0
                # fill the map's current grid with pure white color
                x = i * (img_size*3 + line_width)
                y = j * (img_size*3 + line_width)
                map[x : x + img_size*3 + line_width*2, y : y + img_size*3 + line_width*2, :] = 1.0
print("texture preferring ratio:", np.sum(stmap == 2) / np.sum(stmap != 0)) # V4 digital twin: 0.447
np.save(folder_path + "Fig1/V4_stmap.npy", stmap) # 0 for non-ROI, 1 for shape preferring, 2 for texture preferring
map = np.fliplr(np.flipud(map)) # top-bottom flip and then left-right flip
size = (img_size*3 + line_width)
map = map[38*size:114*size, 38*size:105*size, :] # only keep the roi part
map_save_path = folder_path + "Fig1/V4_ST.png"
matplotlib.image.imsave(map_save_path, map)
del features, map

In [ ]:
def normalize_to_100(FD, Min, Max):
    # 计算 Imap
    Imap = np.floor((FD - Min) / (Max - Min) * 100)
        # 确保 Imap 中的值不小于1
    Imap = np.where(Imap < 1, 1, Imap)
        # 确保 Imap 中的值不大于100
    Imap = np.where(Imap > 100, 100, Imap)
  
    return Imap/100

texture_num = N1 = 448
shape_num = N2 = 366

Rsp = sio.loadmat(folder_path + "V4DT/ST_PRsp.mat")
PRsp_data = Rsp['PRsp']
Rsp = Rsp['PRsp']
Rsp = Rsp[:N1+N2,:,:]

texture_PRsp = PRsp_data[0:texture_num, :,:]  # 448 textures
shape_PRsp = PRsp_data[texture_num:texture_num + shape_num, :,:]  # 366 shapes
ROI_transpose = np.load(folder_path + "V4DT/ROI.npy").T
roi_indices = np.argwhere(ROI_transpose == 1)  
roi_data = Rsp*ROI_transpose

num_neurons = roi_indices.shape[0]
spersity_1 = np.zeros((128,128))

num =  int((N1 + N2)*0.15)
# num = 25

for i in range(num_neurons):
    response = roi_data[:,roi_indices[i,0],roi_indices[i,1]]
    top_1percent_indices = np.argsort(response)[-num:]

    texture_count_1 = np.sum(top_1percent_indices < N1)
    shape_count_1 = np.sum(top_1percent_indices >= N1)

    # print(texture_count,shape_count) 
    spersity_1[roi_indices[i,0],roi_indices[i,1]] = texture_count_1/num

V4FD = sio.loadmat(folder_path + "V4DT/Dispersity_results/FDraw.mat")['FD']
V4FD = normalize_to_100(V4FD, 0, 1)

spersity_nan_1 = np.where(ROI_transpose>0,spersity_1,np.nan)
spersity_nan_flat = spersity_nan_1.flatten()
spersity_nan_1 = np.flip(spersity_nan_1)

plt.imshow(spersity_nan_1,cmap='Spectral')
plt.colorbar()
# plt.show()
# plt.imsave('texture_shape_map.png', spersity_nan_1, cmap='Spectral',vmin=0,vmax=1)

spersity_flat = spersity_1.flatten()
V4FD_flat = V4FD.flatten()
valid_mask = ~np.isnan(spersity_nan_flat)

r, p_value = stats.pearsonr(spersity_flat[valid_mask], V4FD_flat[valid_mask])
print(f"Pearson r: {r}, p-value: {p_value}")
print(f"cosine similarity: {np.dot(spersity_flat[valid_mask],V4FD_flat[valid_mask])/(np.linalg.norm(spersity_flat[valid_mask])*np.linalg.norm(V4FD_flat[valid_mask]))}")

# Simulations

In [ ]:
class SOM():
    """
    2-D Self-Organizing Map with Gaussian Neighbourhood function and linearly decreasing learning rate.
    """
    def __init__(self, m, n, dim, niter, alpha=None, sigma=None, decay_rate=0.5, theta=1, tqdm_disable=False):
        self.device = torch.device('mps' if torch.backends.mps.is_built() else 'cpu')
        self.m = m
        self.n = n
        self.dim = dim
        self.niter = niter
        self.alpha = float(alpha) if alpha is not None else 0.5
        self.sigma = float(sigma) if sigma is not None else max(m, n) / 60.0 # given m = n = 60, default sigma = 5
        self.decay_rate = decay_rate # learning rate decay rate
        self.decay_steps = 10
        self.tqdm_disable = tqdm_disable

        self.positions = False
        self.theta = theta if theta is not None else 1
        self.positions_weighted = torch.cat((torch.full((50000,), self.theta), torch.full((2,), 125.0))).to(self.device)
        self.normalize = False

        # Initialize and normalize weights
        self.weights = torch.nn.functional.normalize(torch.rand((m * n, dim), device=self.device), p=2, dim=1)
        self.locations = torch.tensor(list(self.neuron_locations()), device=self.device)
        self.pdist = nn.PairwiseDistance(p=2).to(self.device)

    def neuron_locations(self):
        for i in range(self.m):
            for j in range(self.n):
                yield [i, j]

    def forward(self, x, it):
        """
        x = x.to(self.device) # to be normalized
        if self.normalize: # normalize input vector's response to 50k images only to give a unit vector with l2 norm = 1
            if self.positions:
                x[:-2] = torch.nn.functional.normalize(x[:-2], p=2, dim=0)
            else:
                x = torch.nn.functional.normalize(x, p=2, dim=0)
        """
        # pairwise distances between input vector and all som neurons: torch.Size([m * n])
        if self.positions:
            stacked = torch.stack([x for i in range(self.m*self.n)]) * self.positions_weighted
            dists = self.pdist(stacked, self.weights)
        else:
            stacked = torch.stack([x for i in range(self.m*self.n)])
            dists = self.pdist(stacked, self.weights)
        _, bmu_index = torch.min(dists, 0)
        bmu_loc = self.locations[bmu_index,:].squeeze()
        
        # decreasing lr step wise over iterations
        # alpha_op = self.alpha * self.decay_rate ** (it // self.decay_steps) # 1.0 - it / self.niter; try with no constant lr for all iterations
        alpha_op = self.alpha # constant lr
        sigma_op = self.sigma - (self.decay_rate * (it // self.decay_steps))
        sigma_op = 1.0 if sigma_op <= 1.0 else sigma_op

        # distance calculation
        bmu_distance_squares = torch.sum(torch.pow(self.locations.float() - torch.stack([bmu_loc for i in range(self.m*self.n)]).float(), 2), 1)
        neighbourhood_func = torch.exp(torch.neg(torch.div(bmu_distance_squares, sigma_op**2)))
        learning_rate_op = alpha_op * neighbourhood_func

        learning_rate_multiplier = torch.stack([learning_rate_op[i:i+1].repeat(self.dim) for i in range(self.m*self.n)])
        delta = torch.mul(learning_rate_multiplier, (stacked - self.weights))
        self.weights = torch.add(self.weights, delta)

    def train(self, data):
        data = torch.tensor(data, device=self.device, dtype=torch.float32)
        if self.normalize:
            for i in tqdm(range(data.shape[0]), desc="Normalizing Data 1st 50k entries..."):
                if self.positions:
                    data[i, :-int(data.shape[1] - 50000)] = torch.nn.functional.normalize(data[i, :-int(data.shape[1] - 50000)], p=2, dim=0)
                else:
                    data[i, :] = torch.nn.functional.normalize(data[i, :], p=2, dim=0)
        for iteration in range(self.niter):
            for i in tqdm(range(data.shape[0]), desc=f"{1+iteration}th Iteration in Progress..."):
                self.forward(data[i, :], iteration)

# Create a V4twin_rsp matrix of shape (3048, 50000) for SOM training
def load_data(position=False): 
    response = np.load(folder_path + "V4DT/PRsp.npy") # (50000, 128, 128)
    roi = np.load(folder_path + "V4DT/ROI.npy").T # (128, 128) # Transpose is crucial! This matches the "roi" with "response"
    if position:
        theta = sio.loadmat(folder_path + "V4DT/RF_results/theta_map_raw.mat")['theta_map_raw'] # (128, 128)
        r = sio.loadmat(folder_path + "V4DT/RF_results/r_map_raw.mat")['r_map_raw'] # (128, 128)
        # Initialize data array: (N, 50000 + 2)
        # The extra 2 columns are for 'r' and 'theta'
        data = np.zeros((np.sum(roi), response.shape[0] + 2))
        
        entry = 0
        for i in range(roi.shape[0]):
            for j in range(roi.shape[1]):
                if roi[i, j] == 1:
                    # 1. Copy the 50k response features
                    data[entry, :-2] = response[:, i, j]
                    
                    # 2. Append the position features (r, theta)
                    data[entry, -2] = r[i, j]
                    data[entry, -1] = theta[i, j]
                    
                    entry += 1
    else:
        data = np.zeros((np.sum(roi), response.shape[0]))
        entry = 0
        for i in range(roi.shape[0]):
            for j in range(roi.shape[1]):
                if roi[i, j] == 1:
                    data[entry, :] = response[:, i, j]
                    entry += 1
    assert entry == data.shape[0]
    return data

In [ ]:
som = SOM(60, 60, 50000, 120, alpha=0.5, sigma=5, decay_rate=0.5, theta=1, tqdm_disable=False)
som.train(load_data(position=False))
som_weights = som.weights.cpu().detach().numpy()
print("SOM training completed, with SOM weight:", som_weights.shape)

In [ ]:
rsom = SOM(60, 60, 50002, 120, alpha=0.5, sigma=5, decay_rate=0.5, theta=1, tqdm_disable=False)
rsom.positions = True # Important: Enable the position flag manually
rsom.train(load_data(position=True))
rsom_weights = rsom.weights.cpu().detach().numpy()
print("RSOM training completed, with RSOM weight:", rsom_weights.shape)

# RSOM image preference map

In [ ]:
name = "RSOM"  # or "SOM", folder to the trained RSOM or SOM model
features = np.load(folder_path + name + "/weights.npy") # (60, 60, 50000(+))
if features.shape[2] > 50000:
    features = features[:, :, :-int(features.shape[2] - 50000)] # remove positional information from the data
    assert features.shape[2] == 50000
# define the size of a single image
grid_num = 60
img_size = 30
line_width = 5
top_img_num = 9
roi = np.ones((grid_num, grid_num)) # all grids are roi
# create a blank map of black color (R=0, G=0, B=0)
map = np.zeros((grid_num * (img_size*3 + line_width) + line_width,
                grid_num * (img_size*3 + line_width) + line_width, 
                3))
grid_top_images = np.zeros((grid_num, grid_num, top_img_num)) # store the top 9 images of each grid
# fill the map with the images
for i in tqdm(range(grid_num), desc="map initialization...", disable=False):
    for j in range(grid_num):
            if roi[i, j] == 1:
                # 1-indexed image names
                image_label = np.arange(50000) + 1 # or "labels + 1" if features were selected from the top-3 responsive images
                # sort the mean responses (from small to large) and the image_label according to the order of mean responses
                _, image_label = zip(*sorted(zip(features[i, j, :],image_label)))
                # take the top nine images with largest mean response
                image_label = np.flip(image_label[-top_img_num:])
                # store the 0-indexed 9 imgs of each grid
                grid_top_images[i, j, :] = (image_label - 1).astype(int)
                # locate the top left corner of the current grid in the map, map transposed, map is transposed here
                x = j * (img_size*3 + line_width) + line_width
                y = i * (img_size*3 + line_width) + line_width
                # fill the map's current grid with the selected nine images
                for row in range(3):
                    for col in range(3): 
                        # load the image
                        path = folder_path + "50K_Imgset/" + str(int(image_label[row*3+col])) + ".bmp" # the image name is 1-indexed
                        img = np.array(Image.open(path))[20:80, 20:80, :] # obtain the non-blurred central part of the image
                        img = resize(img, (img_size, img_size, 3), anti_aliasing=True) # resize the image
                        # put the image onto the map
                        map[x + (2 - row) * img_size : x + ((2 - row) + 1) * img_size, 
                            y + (2 - col) * img_size : y + ((2 - col) + 1) * img_size, 
                            :] = img
np.save(folder_path + name + "/rsptop_0index.npy", grid_top_images)
matplotlib.image.imsave(folder_path + "Fig1/RSOM.jpg", map)

# Dispersity

In [ ]:
# load dispersity
dispersity = sio.loadmat(folder_path + "V4DT/Dispersity_results/dispersity.mat")['cmap'] # (128, 128, 3) <class 'numpy.ndarray'>
dispersity_raw = sio.loadmat(folder_path + "V4DT/Dispersity_results/FDraw.mat")["FD"]

# load roi
roi = np.load(folder_path + "V4DT/ROI.npy").T # (128, 128) <class 'numpy.ndarray'>

# load predicted responses to both texture and shape images
PRsp = sio.loadmat(folder_path + "V4DT/ST_PRsp.mat")["PRsp"]
texture_num = 448
shape_num = 366
texture_PRsp = PRsp[0:texture_num, :,:]  # texture imgs, (448, 128, 128)
texture_PRsp = np.mean(texture_PRsp, axis=0)
texture_PRsp[np.where(roi!=1)] = 0
shape_PRsp = PRsp[texture_num:texture_num + shape_num, :,:]  # shape imgs, (366, 128, 128)
shape_PRsp = np.mean(shape_PRsp, axis=0)
shape_PRsp[np.where(roi!=1)] = 0
ts_ratio = (texture_PRsp + 1) / (shape_PRsp + 1) - 1 # texture_mean_response / shape_mean_response

# visualization
fig, axes = plt.subplots(1, 6, figsize=(12, 2))
axes[0].imshow(np.flip(roi[10:100, 20:95]))
axes[0].axis('off')  # Turn off the axis
axes[0].set_title('ROI')
axes[1].imshow(np.flip(texture_PRsp[10:100, 20:95]))
axes[1].axis('off')  # Turn off the axis
axes[1].set_title('texture')
axes[2].imshow(np.flip(shape_PRsp[10:100, 20:95]))
axes[2].axis('off')  # Turn off the axis
axes[2].set_title('shape')
axes[3].imshow(np.flip(ts_ratio[10:100, 20:95]))
axes[3].axis("off")
axes[3].set_title('texture/shape')
axes[4].imshow(np.flip(dispersity, axis=(0, 1))[30:120, 35:110, :])
axes[4].axis('off')  # Turn off the axis
axes[4].set_title('dispersity')
axes[5].imshow(np.flip(dispersity_raw)[30:120, 35:110])
axes[5].axis('off')  # Turn off the axis
axes[5].set_title('dispersity_raw')
plt.tight_layout()
plt.show()

In [ ]:
# visualize V4 feature dispersity map
d = dispersity
d[roi!=1] = 1
plt.imshow(np.flip(d, axis=(0, 1))[30:120, 35:110, :])
plt.axis('off')  # Turn off the axis
plt.tight_layout()
plt.savefig(folder_path + "Fig1/dispersity.png", dpi=1000)
plt.close()

In [ ]:
# Example images associated with different feature dispersity values
d = dispersity_raw[roi!=0].flatten()
np.random.seed(4)
segments = np.linspace(0, 1, 17)
sampled_indices = []
fds = []
for i in range(len(segments) - 1):
    lower, upper = segments[i], segments[i + 1] # Find indices where values are within the current segment
    indices = np.argwhere((d >= lower) & (d < upper)) # If there are valid indices, randomly select one
    if len(indices) > 0:
        chosen_index = indices[np.random.choice(len(indices))]
        sampled_indices.append(chosen_index)
        fds.append(float(d[chosen_index]))
# Convert sampled indices to a numpy array for convenience
sampled_indices = np.array(sampled_indices)
fds = np.array(fds)
print("chosen feature dispersity values:", fds)

img_0index = np.load(folder_path + "V4DT/top9_0index.npy")
img_0index = img_0index[roi!=0] # (3048, 9)
# create a blank map of black color (R=0, G=0, B=0)
unit_num = len(sampled_indices)
img_size = 60
line_width = 3
img_num = 7 # top imgs to be displayed
sr = 10 # surround width
map = np.zeros((img_num * img_size + (img_num - 1) * line_width + 2 * sr,
                unit_num * img_size + (unit_num - 1) * line_width + 2 * sr, 
                3))
# Fill the map with the images
for i in range(len(sampled_indices)):
    idx = sampled_indices[i]
    imgs = img_0index[idx, :img_num]
    for j in range(img_num):
        path = folder_path + "50K_Imgset/" + str(int(imgs[0][j] + 1)) + ".bmp" # the image name is 1-indexed
        img = np.array(Image.open(path))[20:80, 20:80, :] # obtain the non-blurred central part of the image
        img = resize(img, (img_size, img_size, 3), anti_aliasing=True) # resize the image
        # img = np.fliplr(np.flipud(img)) # flip the image vertically and horizontally
        map[(sr + j * (img_size + line_width)) : (sr + j * (img_size + line_width) + img_size),
            (sr + i * (img_size + line_width)) : (sr + i * (img_size + line_width) + img_size), 
            :] = img

plt.imshow(map)
plt.axis('off')  # Turn off the axis
plt.tight_layout()
plt.savefig(folder_path + "Fig1/fd_img.png", dpi=100)

# Tuning curve shape

In [ ]:
# tuning curve comparison figure
# Compare the tuning curves of V4 digital twin benchmark and the learned SOM
# Load the V4 digital twin responses to 50k images
response = np.load(folder_path + "V4DT/PRsp.npy")
roi = np.load(folder_path + "V4DT/ROI.npy").T # (128, 128) <class 'numpy.ndarray'>
v4_benchmark = response[:, roi == 1] # (50000, 128, 128) into (50000, 3048)
print(v4_benchmark.shape) # (50000, 3048)
v4_benchmark = np.sort(v4_benchmark, axis=0)[::-1]  # Sort each column along rows (flip for descending order)
print(v4_benchmark.shape) # (50000, 3048)
v4_mean = np.mean(v4_benchmark, axis=1) # mean tuning curve
v4_std = np.std(v4_benchmark, axis=1) # tuning curve std
index = []
for i in range(v4_benchmark.shape[1]):
    assert len(v4_benchmark[:, i]) == 50000
    half = (np.max(v4_benchmark[:, i]) - np.min(v4_benchmark[:, i])) / 2
    for j in range(50000):
        if v4_benchmark[j, i] <= half:
            index.append(j)
            break
print("V4 half:", np.mean(index), np.std(index))

name_all = ["RSOM", "SOM"]
for name in name_all:
    rsp = np.load(folder_path + name + "/weights.npy")
    if rsp.shape[2] > 50000: rsp = rsp[:, :, :-int(rsp.shape[2] - 50000)] # now rsp / som weights has shape (60, 60, 50000)
    rsp = rsp.reshape(-1, rsp.shape[2]) # (3600, 50000)
    rsp = np.sort(rsp.T, axis=0)[::-1]  # Sort each column along rows (flip for descending order), (50000, 3600)
    rsp_mean = np.mean(rsp, axis=1) # mean tuning curve
    rsp_std = np.std(rsp, axis=1) # tuning curve std
    # normalize every tuning curve
    index = []
    for i in range(rsp.shape[1]):
        assert len(rsp[:, i]) == 50000
        half = (np.max(rsp[:, i]) - np.min(rsp[:, i])) / 2
        for j in range(50000):
            if rsp[j, i] <= half:
                index.append(j)
                break
    # visualization
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    print("V4 and simulated tuning curve correlation: {:.3f}".format(pearsonr(np.concatenate((v4_mean, v4_std)), np.concatenate((rsp_mean, rsp_std)))[0]))
    axes[0].plot(v4_mean, color='black')
    axes[0].fill_between(range(len(v4_mean)), v4_mean - v4_std, v4_mean + v4_std, color='black', alpha=0.3)
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    axes[0].set_xlabel('50K imgs', fontsize=16)
    axes[0].set_ylabel('response', fontsize=16)
    axes[0].set_title('V4', fontsize=18)
    axes[1].plot(rsp_mean, color='gray')
    axes[1].fill_between(range(len(rsp_mean)), rsp_mean - rsp_std, rsp_mean + rsp_std, color='gray', alpha=0.3)
    axes[1].set_xticks([])
    axes[1].set_yticks([])
    axes[1].set_xlabel('50K imgs', fontsize=16)
    axes[1].set_title("simulation", fontsize=18)
    fig = plt.gcf()
    plt.tight_layout()
    plt.close()
    fig.savefig(folder_path + name + "/tunings.png", dpi=1000)
    del rsp, rsp_mean, rsp_std
print("number of images at half maximum activation value", np.mean(index), np.std(index))
del v4_benchmark, v4_mean, v4_std, name_all, name